In [3]:
search_for = 128
start_from = 0
threads = 10
thread = 5
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.01703166961669922
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 330
all cases: 866251507
type: [1, 1, 1, 1, 128] 128
cases of this type: 268435456
100000 37.751317636749015
200000 64.38327656901994
300000 68.22865107475317
400000 66.2064868455487
500000 66.22664346478354
600000 69.4660450009086
700000 65.90884105378592
800000 65.32951004112711
900000 65.4977070315731
1000000 65.42144860079803
1100000 20.51056712004028
1200000 66.95338615355263
1300000 65.05701590266268
1400000 64.96765956354307
1500000 64.1342260950575
1600000 68.33010835729725
1700000 65.42014352853556
1800000 64.72781646360248
1900000 66.23173801755173
2000000 65.40450288949458
2100000 51.664877252847695
2200000 35.48042694494851
2300000 64.03713764205345
2400000 64.04795683006122
2500000 63.01677859217551
2600000 63.83415298269688
2700000 65.95714892261913
2800000 63.73499083189671
2900000 63.24442971072873
3000000 63.1322734277809
3100

30100000 69.09252515581721
30200000 72.65785404431203
30300000 66.45489197151743
30400000 57.9870591324925
30500000 37.44591232583247
30600000 66.19192811915532
30700000 69.69522832519507
30800000 68.68439803689819
30900000 73.68644876547003
31000000 71.29863841065291
31100000 74.93104887725615
31200000 71.49164990769557
31300000 40.245896301311625
31400000 71.93570080753047
31500000 78.16969603197958
31600000 75.68955383444859
31700000 64.5947030162519
31800000 45.694265915404785
31900000 77.65911976028832
32000000 74.87251014233095
32100000 76.16627446314865
32200000 71.12279356532365
32300000 74.77256169200336
32400000 67.31426188349786
32500000 58.77943402176729
32600000 40.141199270305776
32700000 66.87937952779242
32800000 71.31954245597124
32900000 70.771945034817
33000000 72.22501363155071
33100000 76.70297990431112
33200000 74.69001896272597
33300000 66.74578908294559
33400000 45.646459365150164
33500000 77.74381123151609
33600000 76.55423955159787
33700000 77.44215421832256
3

60600000 78.62458123177824
60700000 75.58272873226565
60800000 78.12673144427674
60900000 76.98188274353117
61000000 78.46109132235281
61100000 74.59389491413314
61200000 77.05494802237574
61300000 50.311730590565716
61400000 50.001859900390635
61500000 71.01238283346868
61600000 64.50316100681954
61700000 66.22525906199749
61800000 70.25885157173398
61900000 30.678499901554858
62000000 67.62734597583852
62100000 68.25062153473158
62200000 65.92463590652991
62300000 68.969111523674
62400000 70.9605428811416
62500000 35.12317086385317
62600000 76.5950170073312
62700000 76.01254164212173
62800000 75.31956192130379
62900000 79.67535144355475
63000000 75.78793052060787
63100000 77.64786544462811
63200000 74.79251477615806
63300000 72.82577336259916
63400000 58.54748095066502
63500000 44.54710730812855
63600000 67.7572340545712
63700000 64.3656119799846
63800000 67.93602628732494
63900000 66.27992406141375
64000000 28.11463621617736
64100000 64.1603449003233
64200000 69.76210634457033
64300

91000000 53.48967986105125
91100000 63.317351434807826
91200000 60.46935005643765
91300000 24.722463952924684
91400000 63.41126654471565
91500000 63.25749449861177
91600000 39.166787701755005
91700000 61.911035228404245
91800000 65.71915400336269
91900000 66.39091201636681
92000000 68.3442721675148
92100000 69.57021137025195
92200000 73.3234091631956
92300000 67.4725151370319
92400000 66.93668162639324
92500000 68.00726529773995
92600000 70.78191986301002
92700000 66.94874404976575
92800000 67.82052277782344
92900000 64.11968942875671
93000000 47.99541544599748
93100000 43.43252129881178
93200000 64.85179392344818
93300000 57.9323701550194
93400000 24.65239370430956
93500000 62.96425063731507
93600000 62.093077721884576
93700000 29.497429883557178
93800000 68.54364226447721
93900000 64.63070907242336
94000000 63.641236795263424
94100000 65.76405548300812
94200000 63.20927593950453
94300000 67.44751635347666
94400000 69.09081846409158
94500000 70.08488228577157
94600000 72.4908383418974

120500000 34.07128439439557
120600000 28.452885891169988
120700000 30.485172241153155
120800000 46.86274601918188
120900000 61.684874762526825
121000000 62.288705727993
121100000 60.0962754239534
121200000 61.47467092115527
121300000 60.53402607913581
121400000 61.45201542462532
121500000 62.13631997946086
121600000 68.42070671304614
121700000 69.15635028972818
121800000 63.25276248509799
121900000 60.049639891787805
122000000 65.68708419991208
122100000 56.84651637426044
122200000 56.14458744466645
122300000 63.043784664919144
122400000 57.029167581640266
122500000 60.332019107826895
122600000 42.43177981472191
122700000 24.77722101395908
122800000 22.780162229142878
122900000 53.54364343002379
123000000 61.47685033367889
123100000 60.33420193403614
123200000 59.10044172699912
123300000 59.001410391486196
123400000 61.22318038235781
123500000 58.99666069725665
123600000 61.92114635526598
123700000 64.30048217858878
123800000 69.554978131195
123900000 61.6968298341024
124000000 61.0499

...

best so far: 0
type: [1, 2, 16, 1, 4] 128
cases of this type: 131072
799800000 0.22443625784807075
799900000 0.2504396693977893
best so far: 0
type: [1, 2, 16, 2, 2] 128
cases of this type: 65536
best so far: 0
type: [1, 2, 16, 4, 1] 128
cases of this type: 32768
800000000 0.24562989521480336
best so far: 0
type: [1, 2, 32, 1, 2] 128
cases of this type: 32768
best so far: 0
type: [1, 2, 32, 2, 1] 128
cases of this type: 16384
best so far: 0
type: [1, 2, 64, 1, 1] 128
cases of this type: 8192
best so far: 0
type: [1, 4, 1, 1, 32] 128
cases of this type: 4194304
800100000 0.49066828993537026
800200000 1.2884262081723075
800300000 1.2889947830838218
800400000 0.993773727301369
800500000 0.9352773554392219
800600000 0.958470400186138
800700000 0.9379620772201075
800800000 0.8555675748653333
800900000 0.9404079069337328
801000000 0.9370431455901735
801100000 0.7945638161801735
801200000 0.9532583603339395
801300000 1.0241677177741726
801400000 1.0807167597447336
801500000 0.7765005571900309

813300000 0.9622105450973047
813400000 0.9748918497266253
813500000 0.8337536771260924
813600000 0.7948934807189258
813700000 0.8681411246025482
813800000 1.0010276646043275
813900000 1.0892329765697817
814000000 0.7980486930175066
814100000 1.1667063371748487
814200000 0.9232930726538637
814300000 0.8658015078655104
814400000 0.7942997583072456
814500000 0.802112840011557
814600000 0.8783142336917004
814700000 0.910152256755189
814800000 0.8080735197007405
814900000 0.8962353834404051
815000000 0.7838595988009348
815100000 0.7931146492892099
815200000 0.877222838119481
815300000 0.9077607133551518
815400000 0.8892644208930692
815500000 0.8322145092204174
815600000 0.7746013939268537
815700000 0.8571311747794019
815800000 0.7037041087483585
815900000 0.8452872207514479
816000000 0.8442493928049127
816100000 0.7205801887951812
816200000 0.8485750460980064
816300000 0.8982248286639354
816400000 0.6715270053634385
816500000 0.9408250861297878
816600000 0.885901657576541
816700000 0.806310

841000000 0.13612678651594518
841100000 0.2565390535296566
841200000 0.2138311290342927
841300000 0.16739298789330917
841400000 0.23259646539760034
841500000 0.16287923774181934
841600000 0.2190527406249702
841700000 0.23287602853161693
841800000 0.1425436346224924
best so far: 0
type: [2, 1, 1, 8, 8] 128
cases of this type: 2097152
841900000 0.24933799412738678
842000000 0.1885084576431314
842100000 0.15595442956107924
842200000 0.22929553599287272
842300000 0.1571866154266351
842400000 0.23027213334451452
842500000 0.20870484232745967
842600000 0.14508487784705493
842700000 0.20806545593201387
842800000 0.16633322402553558
842900000 0.1581840130244308
843000000 0.20571799043752884
843100000 0.14793166055916745
843200000 0.19066849761233795
843300000 0.18041289609299038
843400000 0.14163335845373473
843500000 0.20468426215183338
843600000 0.1476439410190549
843700000 0.16285772584024072
843800000 0.1947633677777668
843900000 0.13425765282058186
best so far: 0
type: [2, 1, 1, 16, 4] 12

861100000 0.025275811344285807
861200000 0.016191297839922375
861300000 0.032525928503535195
best so far: 0
type: [2, 2, 2, 2, 8] 128
cases of this type: 262144
861400000 0.03378371985010107
861500000 0.028754855316668087
861600000 0.015433563935608333
best so far: 0
type: [2, 2, 2, 4, 4] 128
cases of this type: 131072
861700000 0.02487096933233142
best so far: 0
type: [2, 2, 2, 8, 2] 128
cases of this type: 65536
861800000 0.018968972260340052
best so far: 0
type: [2, 2, 2, 16, 1] 128
cases of this type: 32768
best so far: 0
type: [2, 2, 4, 1, 8] 128
cases of this type: 131072
861900000 0.019163370149732958
best so far: 0
type: [2, 2, 4, 2, 4] 128
cases of this type: 65536
862000000 0.01731742116427223
best so far: 0
type: [2, 2, 4, 4, 2] 128
cases of this type: 32768
best so far: 0
type: [2, 2, 4, 8, 1] 128
cases of this type: 16384
862100000 0.01394980829120146
best so far: 0
type: [2, 2, 8, 1, 4] 128
cases of this type: 32768
best so far: 0
type: [2, 2, 8, 2, 2] 128
cases of this t

866100000 0.0006219153191897605
best so far: 0
type: [8, 1, 1, 2, 8] 128
cases of this type: 32768
best so far: 0
type: [8, 1, 1, 4, 4] 128
cases of this type: 16384
best so far: 0
type: [8, 1, 1, 8, 2] 128
cases of this type: 8192
best so far: 0
type: [8, 1, 1, 16, 1] 128
cases of this type: 4096
best so far: 0
type: [8, 1, 2, 1, 8] 128
cases of this type: 16384
best so far: 0
type: [8, 1, 2, 2, 4] 128
cases of this type: 8192
866200000 0.00019619796036481857
best so far: 0
type: [8, 1, 2, 4, 2] 128
cases of this type: 4096
best so far: 0
type: [8, 1, 2, 8, 1] 128
cases of this type: 2048
best so far: 0
type: [8, 1, 4, 1, 4] 128
cases of this type: 4096
best so far: 0
type: [8, 1, 4, 2, 2] 128
cases of this type: 2048
best so far: 0
type: [8, 1, 4, 4, 1] 128
cases of this type: 1024
best so far: 0
type: [8, 1, 8, 1, 2] 128
cases of this type: 1024
best so far: 0
type: [8, 1, 8, 2, 1] 128
cases of this type: 512
best so far: 0
type: [8, 1, 16, 1, 1] 128
cases of this type: 256
best so 